In [ ]:
import pdfplumber
import os
import re
import pandas as pd
import numpy as np

path = "data/Key_Stats/"
files = os.listdir(path)


In [ ]:
def read_pages(dir):
    with pdfplumber.open(dir) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

def full_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text) # only english
    text = re.sub(r'[\n\t]+', ' ', text) # delete '\n' and '\t'
    text = re.sub(r' +', ' ', text) # delete multiple spaces
    return text

def file_id(file):
    pattern = r'[A-Z]{3,3}[0-9]{3,3}'
    return re.findall(pattern, file)

def fund_type(name):
    pattern_bond = r'bond|yield|debt|fixed|income|return'
    pattern_stock = r'equity|stock|growth'
    pattern_index = r'index|benchmark'
    if re.search(pattern_bond, name):
        return 'bond'
    elif re.search(pattern_stock, name):
        return 'stock'
    elif re.search(pattern_index, name):
        return 'index'
    else:
        return 'other'

In [ ]:
pattern1 = r'[0-9]*\.[0-9]+%'
pattern2 = r'[0-9]+%'
pattern3 = r"nil|no|n\/a"
fee_list = ['subscription', 'switching', 'redemption', 'management', 'trustee', 'performance', 'admin']
#fee_table = pd.DataFrame(columns = ['id'] + fee_list)


for i in range(len(files)):
#for i in range(10):
    doc_dir = path + files[i]
    raw = read_pages(doc_dir)
    text = full_text(raw).lower()
    ID = file_id(files[i])
    
    fee_loc = text.find('what are the fees and charges?')
    text_fee = text[fee_loc:]
    #fee_list = ['subscription', 'switching', 'redemption', 'investment management', 'trustee', 'performance', 'admin']
    #fee_table = pd.DataFrame(columns = ['id'] + fee_list)
    fee_fund = pd.DataFrame(columns = ['id'] + fee_list)
    fee_fund['id'] = ID
    for fee in fee_list:
        loc = text_fee.find(fee)
        if loc == -1:
            fee_fund[fee] = np.nan
        else:
            fee1 = re.findall(pattern1,text_fee[loc:])
            fee2 = re.findall(pattern2,text_fee[loc:])
            fee3 = re.findall(pattern3,text_fee[loc:])

            loc1 = text_fee[loc:].find(fee1[0]) if len(fee1) > 0 else 6000
            loc2 = text_fee[loc:].find(fee2[0]) if len(fee2) > 0 else 6000
            loc3 = text_fee[loc:].find(fee3[0]) if len(fee3) > 0 else 6000
            if loc3 < loc2 and loc3 < loc1:
                fee_fund[fee] = np.nan
            else:
                if loc1 < loc2:
                    fee_fund[fee] = fee1[0]
                else:
                    fee_fund[fee] = fee2[0]
    fee_table = pd.concat([fee_table, fee_fund], axis = 0)

In [ ]:
pattern1 = r'day|daily'
pattern2 = r'week|weekly'
pattern3 = r"month|monthly"
fund_freq = pd.DataFrame(columns=['file_id', 'freq'])


for i in range(len(files)):
#for i in range(100):
    doc_dir = path + files[i]
    raw = read_pages(doc_dir)
    text = full_text(raw).lower()
    ID = file_id(files[i])
    
    freq_loc = text.find('dealing frequency')
    text_freq = text[freq_loc:freq_loc+100]
    #fee_list = ['subscription', 'switching', 'redemption', 'investment management', 'trustee', 'performance', 'admin']
    #fee_table = pd.DataFrame(columns = ['id'] + fee_list)
    freq_fund = pd.DataFrame(columns=['file_id', 'freq'])
    freq_fund['file_id'] = ID
    if re.search(pattern3, text_freq):
        freq_fund['freq'] = 'monthly'
    elif re.search(pattern2, text_freq):
        freq_fund['freq'] = 'weekly'
    elif re.search(pattern1, text_freq):
        freq_fund['freq'] = 'daily'
    
    
    fund_freq = pd.concat([fund_freq, freq_fund], axis=0)